# Combine Monthly EVI data from NASA's LP DAAC into a single dataset

Temporal support: monthly, 2014--2020 <br>
Spaital support: 0.05-deg, global

https://doi.org/10.5067/MODIS/MOD13C2.006 <br>
https://lpdaac.usgs.gov/


In [1]:
import sys
sys.path.insert(0, "../src")

import xarray as xr
import data_utils

In [ ]:
paths_evi = "../data/exp_raw/*.hdf"
ds = xr.open_mfdataset(paths_evi)